# BCI_EEG_Neural_Network

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [4]:
import numpy as np

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [6]:
import mne
from moabb.datasets import BNCI2014_001
from moabb.paradigms import MotorImagery

In [8]:
def get_bci_data(subject_id=1):
    print("Downloading data")

    dataset = BNCI2014_001()
    dataset.subject_list = [subject_id]

    paradigm = MotorImagery(n_classes=4, fmin=8, fmax=32)

    X, y, metadata = paradigm.get_data(dataset=dataset, subjects=[subject_id])

    le = LabelEncoder()
    y = le.fit_transform(y)

    print(f"Shape X {X.shape}, y {y.shape}, classes {le.classes_}")
    return X, y

In [9]:
# define model 

class BCIMLP(nn.Module):
    def __init__(self, input_features, hidden_sizes, output_size, dropout=0.3):
        super(BCIMLP, self).__init__()

        layers = []
        prev_size = input_features

        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout))
            prev_size = hidden_size

        layers.append(nn.Linear(prev_size, output_size))

        self.network = nn.Sequential(*layers)

    def forward(slef, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1)
        return network(x)


In [10]:
# --- Configuration ---
BATCH_SIZE = 32
HIDDEN_SIZES = [512, 256]  # Two hidden layers (was HIDDEN_SIZE in RNN)
LEARNING_RATE = 0.001
EPOCHS = 50
DEVICE = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [11]:
DEVICE

device(type='mps')

In [12]:
X_raw, y_raw = get_bci_data()

Choosing from all possible events


Shape X (576, 22, 1001), y (576,), classes ['feet' 'left_hand' 'right_hand' 'tongue']


In [13]:
# Normalize features per channel
N, C, T = X_raw.shape
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw.reshape(-1, C)).reshape(N, C, T)

In [14]:
# convert to tensor
X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(DEVICE)
y_tensor = torch.tensor(y_raw, dtype=torch.long).to(DEVICE)